In [ ]:
!pip install flask

In [ ]:
!pip install vllm torch pyngrok

In [ ]:
!ngrok config add-authtoken  YOUR_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import json
import torch
import re
from vllm import LLM, SamplingParams
from pyngrok import ngrok
from flask import Flask, request, jsonify

# GPU kontrolü
print(f"CUDA mevcut: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Bellek: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Model 1: Task Planlayıcı (DPO)
print("Model 1 yükleniyor (Task Planner)...")
llm_planner = LLM(
    model="ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
    trust_remote_code=True,
    gpu_memory_utilization=0.45,
    max_model_len=4096
)
print("✅ Model 1 yüklendi!")

# Model 2: JSON Dönüştürücü (Instruct)
print("Model 2 yükleniyor (JSON Converter)...")
llm_converter = LLM(
    model="ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1",
    trust_remote_code=True,
    gpu_memory_utilization=0.45,
    max_model_len=4096
)
print("✅ Model 2 yüklendi!")

# ============= PARSE FONKSİYONLARI =============
def parse_turkish_date(date_str):
    """Türkçe tarihi YYYY-MM-DD formatına çevir"""
    months = {
        "ocak": "01", "şubat": "02", "mart": "03", "nisan": "04",
        "mayıs": "05", "haziran": "06", "temmuz": "07", "ağustos": "08",
        "eylül": "09", "ekim": "10", "kasım": "11", "aralık": "12"
    }

    # "15 Kasım 2025" formatını parse et
    match = re.search(r'(\d+)\s+(\w+)\s+(\d{4})', date_str, re.IGNORECASE)
    if match:
        day = match.group(1).zfill(2)
        month = months.get(match.group(2).lower(), "01")
        year = match.group(3)
        return f"{year}-{month}-{day}"
    return "2025-12-31"  # Default

def parse_time_estimate(estimate_str):
    """Süre tahminini Jira formatına çevir"""
    estimate_str = estimate_str.lower()
    if "gün" in estimate_str:
        days = re.search(r'(\d+)', estimate_str)
        if days:
            return f"{days.group(1)}d"
    elif "saat" in estimate_str:
        hours = re.search(r'(\d+)', estimate_str)
        if hours:
            return f"{hours.group(1)}h"
    elif "hafta" in estimate_str:
        weeks = re.search(r'(\d+)', estimate_str)
        if weeks:
            return f"{int(weeks.group(1)) * 5}d"
    return "1d"

def parse_priority(priority_str):
    """Önceliği İngilizce'ye çevir"""
    priority_map = {
        "yüksek": "High",
        "orta": "Medium",
        "düşük": "Low",
        "çok yüksek": "Highest",
        "çok düşük": "Lowest"
    }
    return priority_map.get(priority_str.lower(), "Medium")

def create_epic_key(epic_name):
    """Epic isminden key oluştur"""
    # Türkçe karakterleri değiştir
    replacements = {
        'ş': 's', 'Ş': 'S', 'ı': 'i', 'İ': 'I', 'ğ': 'g', 'Ğ': 'G',
        'ü': 'u', 'Ü': 'U', 'ö': 'o', 'Ö': 'O', 'ç': 'c', 'Ç': 'C'
    }

    for tr_char, en_char in replacements.items():
        epic_name = epic_name.replace(tr_char, en_char)

    # Boşlukları tire ile değiştir, özel karakterleri kaldır
    key = re.sub(r'[^a-zA-Z0-9\s-]', '', epic_name)
    key = re.sub(r'\s+', '-', key.strip())
    return key.upper()

def parse_task_block(task_text):
    """Tek bir task bloğunu parse et"""
    lines = task_text.strip().split('\n')

    task_data = {
        'summary': '',
        'description': '',
        'assignee_name': '',
        'assignee_id': '',
        'priority': 'Medium',
        'estimate': '1d',
        'deadline': '2025-12-31',
        'labels': []
    }

    # Task başlığını al (### Task1: Başlık)
    for line in lines:
        if line.startswith('###'):
            task_data['summary'] = line.split(':', 1)[1].strip() if ':' in line else line.replace('###', '').strip()
            break

    # Diğer alanları parse et
    for line in lines:
        line_lower = line.lower().strip()

        if line_lower.startswith('açıklama:'):
            task_data['description'] = line.split(':', 1)[1].strip()
        elif line_lower.startswith('atama:'):
            assignee_text = line.split(':', 1)[1].strip()
            # "Ali Yılmaz (e01)" veya "Ali Yılmaz" formatını parse et
            match = re.search(r'(.+?)\s*\(([^)]+)\)', assignee_text)
            if match:
                task_data['assignee_name'] = match.group(1).strip()
                task_data['assignee_id'] = match.group(2).strip()
            else:
                # Parantez yoksa sadece ismi al
                task_data['assignee_name'] = assignee_text.strip()
                task_data['assignee_id'] = ''
        elif line_lower.startswith('öncelik:'):
            priority_text = line.split(':', 1)[1].strip()
            task_data['priority'] = parse_priority(priority_text)
        elif line_lower.startswith('tahmin:'):
            estimate_text = line.split(':', 1)[1].strip()
            task_data['estimate'] = parse_time_estimate(estimate_text)
        elif line_lower.startswith('deadline:'):
            deadline_text = line.split(':', 1)[1].strip()
            task_data['deadline'] = parse_turkish_date(deadline_text)
        elif line_lower.startswith('etiketler:'):
            labels_text = line.split(':', 1)[1].strip()
            task_data['labels'] = [label.strip() for label in labels_text.split(',') if label.strip()]

    return task_data

def convert_markdown_to_jira_json(project_key, markdown_text):
    """Markdown formatındaki Epic/Task'ları Jira JSON'a çevir"""
    tasks = []
    current_epic = None
    current_task_text = []

    lines = markdown_text.split('\n')

    for line in lines:
        # Epic başlığı (## Epik 1: ...)
        if line.startswith('## '):
            # Önceki task'ı kaydet
            if current_task_text and current_epic:
                task_data = parse_task_block('\n'.join(current_task_text))
                tasks.append({
                    'epic_name': current_epic,
                    'task_data': task_data
                })
            current_task_text = []

            # Yeni epic'i kaydet
            current_epic = line.replace('##', '').strip()
            if ':' in current_epic:
                current_epic = current_epic.split(':', 1)[1].strip()

        # Task başlığı (### Task1: ...)
        elif line.startswith('### '):
            # Önceki task'ı kaydet
            if current_task_text and current_epic:
                task_data = parse_task_block('\n'.join(current_task_text))
                tasks.append({
                    'epic_name': current_epic,
                    'task_data': task_data
                })
            current_task_text = [line]

        # Task içeriği
        elif current_task_text:
            current_task_text.append(line)

    # Son task'ı kaydet
    if current_task_text and current_epic:
        task_data = parse_task_block('\n'.join(current_task_text))
        tasks.append({
            'epic_name': current_epic,
            'task_data': task_data
        })

    # Jira JSON formatına çevir
    jira_tasks = []
    for task in tasks:
        epic_key = create_epic_key(task['epic_name'])
        task_data = task['task_data']

        # Assignee objesi oluştur (hem name hem accountId)
        assignee_obj = {'name': task_data['assignee_name']}
        if task_data['assignee_id']:
            assignee_obj['accountId'] = task_data['assignee_id']

        jira_task = {
            'epic_name': task['epic_name'],
            'fields': {
                'project': {'key': project_key},
                'issuetype': {'name': 'Task'},
                'summary': task_data['summary'],
                'description': {
                    'type': 'doc',
                    'version': 1,
                    'content': [
                        {
                            'type': 'paragraph',
                            'content': [
                                {'type': 'text', 'text': task_data['description']}
                            ]
                        }
                    ]
                },
                'assignee': assignee_obj,
                'priority': {'name': task_data['priority']},
                'duedate': task_data['deadline'],
                'labels': task_data['labels'],
                'parent': {'key': epic_key},
                'timetracking': {'originalEstimate': task_data['estimate']}
            }
        }
        jira_tasks.append(jira_task)

    return {'tasks': jira_tasks}

# ============= AŞAMA 1: TASK PLANLAYICI =============
# ============= AŞAMA 1: TASK PLANLAYICI =============
SYSTEM_PROMPT_PLANNER = """Sen, "Jira Co-Pilot" adlı, son derece analitik ve tecrübeli bir Kıdemli Proje Yöneticisi Asistanısın.
Görevin, verilen proje dokümanlarını ve ekip listesini okuyup, projeyi mantıklı iş paketlerine (Görevlere) ayırmaktır.

ÇIKTI FORMATI:
1. Projeyi önce ana iş bölümlerine Epikler olarak ayır. Her Epik için Markdown başlığı kullan (## Epik 1: …)
2. Her Epik başlığı altına görevleri Task1, Task2… olarak listele (### Task1: …)
3. Her görev başlığı altına kritik analiz paragrafı yaz:
    - Açıklama (Description)
    - Atama (Assignee)
    - Öncelik (Priority)
    - Tahmin (Estimate)
    - Deadline / Tarih (Deadline)
    - Etiketler (Labels)

KURALLAR:
* Her görev için tüm analiz bilgilerini paragrafın içine doğal bir dille yerleştir.
* Epik ve Task sıralaması mantıklı olmalı; numaralandırmalar karışmamalı.
* Deadline'lar, projenin kritik yolunu ve önceliklerini dikkate alarak mantıklı şekilde atanmalı.
* **Görev atamalarını (Assignee) yaparken, sana verilen ekip listesindeki kişilerin 'Yetkinlikler' (Skills) ve 'Rol' (Role) bilgilerini dikkate al. Görevi, o işe en uygun yetkinlikteki kişiye ata.**
* **'Atama (Assignee)' alanını yazarken, kişinin adıyla birlikte parantez içinde `employee_id`'sini (örn: `Ali Yılmaz (e01)` veya `Ayşe Demir (e02)`) MUTLAKA ekle. Bu, JSON'a dönüştürme için kritik öneme sahiptir ve asla unutulmamalıdır.**

Aşağıdaki proje bilgilerini kullan:"""

# ============= AŞAMA 2: JSON DÖNÜŞTÜRÜCÜ (MODEL-BASED) =============
SYSTEM_PROMPT_CONVERTER = """Sen bir proje yönetimi asistanısın. Görevin, verilen Epic ve Task bilgilerini Jira API formatına çevirmek.

ÖNEMLİ KURALLAR:
1. Çıktı formatı SADECE JSON olmalı, başka hiçbir metin ekleme
2. Her task için "fields" objesi doldur
3. "parent.key" alanına Epic key'ini yaz (Epic başlığından otomatik oluştur: "Görev Ekleme ve Analizi" -> "GOREV-EKLEME-VE-ANALIZI")
4. "description" alanı mutlaka Jira'nın doc formatında olmalı
5. Öncelik çevirisi: Yüksek=High, Orta=Medium, Düşük=Low
6. "duedate" formatı: YYYY-MM-DD (Türkçe tarihten çevir: "15 Kasım 2025" -> "2025-11-15")
7. "timetracking.originalEstimate" formatı: "X gün" -> "Xd", "X saat" -> "Xh", "X hafta" -> "Xw"
8. "assignee" için hem name hem de accountId ekle. AccountId parantez içindeki employee_id değeridir (örn: "Ali Yılmaz (e14)" -> accountId: "e14")
9. "labels" etiketlerden oluştur
10. "summary" task başlığından al
11. JSON array’inin dışına hiçbir şey ekleme. Json biter bitmez Çıktı üretmeyi kes

ÇIKTI FORMATI:
{
  "tasks": [
    {
      "epic_name": "Epic başlığı",
      "fields": {
        "project": {"key": "PROJE_KODU"},
        "issuetype": {"name": "Task"},
        "summary": "Task başlığı",
        "description": {
          "type": "doc",
          "version": 1,
          "content": [
            {
              "type": "paragraph",
              "content": [
                {"type": "text", "text": "Açıklama metni"}
              ]
            }
          ]
        },
        "assignee": {
          "name": "Ali Yılmaz",
          "accountId": "e14"
        },
        "priority": {"name": "High"},
        "duedate": "2025-11-15",
        "labels": ["etiket1", "etiket2"],
        "parent": {"key": "EPIC-KEY"},
        "timetracking": {"originalEstimate": "2d"}
      }
    }
  ]
}"""

def clean_json_response(json_str):
    """JSON yanıtını temizle ve düzelt"""
    # Satır sonlarını ve fazla boşlukları temizle
    json_str = re.sub(r'\s+', ' ', json_str)

    # Trailing comma'ları temizle
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*]', ']', json_str)

    return json_str

def post_process_json(jira_json):
    """
    Model'in ürettiği JSON'daki formatları düzelt
    - Türkçe tarihleri YYYY-MM-DD'ye çevir
    - Süre tahminlerini Jira formatına çevir
    - Öncelikleri İngilizce'ye çevir
    - Assignee formatını düzelt
    """
    if not jira_json or "tasks" not in jira_json:
        return jira_json

    processed_tasks = []

    for task in jira_json.get("tasks", []):
        fields = task.get("fields", {})

        # Tarih düzeltme (eğer Türkçe ise)
        if "duedate" in fields:
            duedate = fields["duedate"]
            if isinstance(duedate, str) and any(month in duedate.lower() for month in ["ocak", "şubat", "mart", "nisan", "mayıs", "haziran", "temmuz", "ağustos", "eylül", "ekim", "kasım", "aralık"]):
                fields["duedate"] = parse_turkish_date(duedate)

        # Süre tahmini düzeltme
        if "timetracking" in fields and "originalEstimate" in fields["timetracking"]:
            estimate = fields["timetracking"]["originalEstimate"]
            if isinstance(estimate, str) and any(word in estimate.lower() for word in ["gün", "saat", "hafta"]):
                fields["timetracking"]["originalEstimate"] = parse_time_estimate(estimate)

        # Öncelik düzeltme (eğer Türkçe ise)
        if "priority" in fields and "name" in fields["priority"]:
            priority = fields["priority"]["name"]
            if isinstance(priority, str) and priority.lower() in ["yüksek", "orta", "düşük", "çok yüksek", "çok düşük"]:
                fields["priority"]["name"] = parse_priority(priority)

        # Assignee düzeltme (parantez içinden accountId çıkar)
        if "assignee" in fields and "name" in fields["assignee"]:
            assignee_name = fields["assignee"]["name"]
            if isinstance(assignee_name, str) and "(" in assignee_name and ")" in assignee_name:
                match = re.search(r'(.+?)\s*\(([^)]+)\)', assignee_name)
                if match:
                    fields["assignee"]["name"] = match.group(1).strip()
                    fields["assignee"]["accountId"] = match.group(2).strip()

        # Epic key düzeltme (Türkçe karakterleri değiştir)
        if "parent" in fields and "key" in fields["parent"]:
            epic_key = fields["parent"]["key"]
            if isinstance(epic_key, str):
                fields["parent"]["key"] = create_epic_key(epic_key)

        processed_tasks.append(task)

    return {"tasks": processed_tasks}

def stage2_convert_to_json_with_model(project_key, task_text, temperature=0.0, max_tokens=8192):

    """AŞAMA 2 (MODEL): Epic/Task metnini model ile Jira JSON'a çevir ve post-process yap"""
    print("\n🔄 AŞAMA 2 (MODEL): JSON Dönüştürücü çalışıyor...")
    torch.cuda.empty_cache()

    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=0.95,
        max_tokens=8192
    )

    user_prompt = f"""Proje Kodu: {project_key}

Aşağıdaki Epic ve Task bilgilerini Jira JSON formatına çevir:

{task_text}

Her Epic için ayrı parent key oluştur. Epic isminden key üret (boşlukları tire ile değiştir, Türkçe karakterleri değiştir).
Sadece JSON çıktısı ver, başka açıklama ekleme."""

    formatted_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM_PROMPT_CONVERTER}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    outputs = llm_converter.generate([formatted_prompt], sampling_params)
    response = outputs[0].outputs[0].text
    print("\n📄 HAM JSON ÇIKTI (parse edilmeden):\n")
    print(response)

    try:
        json_start = response.find('{')

        if json_start == -1:
            return None, "JSON formatı bulunamadı (açılış parantezi yok)"

        # Shunting Yard benzeri parantez sayma algoritması
        json_end = -1
        brace_count = 0
        in_string = False
        escape_next = False

        for i in range(json_start, len(response)):
            char = response[i]

            # Escape karakteri kontrolü
            if escape_next:
                escape_next = False
                continue

            # String içinde mi kontrolü
            if char == '\\':
                escape_next = True
                continue

            if char == '"':
                in_string = not in_string
                continue

            # String dışındayken parantez sayısını takip et
            if not in_string:
                if char == '{':
                    brace_count += 1
                elif char == '}':
                    brace_count -= 1

                    # İlk açılan parantez kapandığında JSON bitmiştir
                    if brace_count == 0:
                        json_end = i + 1
                        print(f"✅ JSON sonu bulundu (pozisyon: {json_end}, toplam {json_end - json_start} karakter)")
                        break

        if json_end == -1:
            return None, "JSON formatı bulunamadı (kapanış parantezi eksik)"

        if json_start != -1 and json_end > json_start:
            json_str = response[json_start:json_end]

            # JSON'ı temizle
            json_str = clean_json_response(json_str)

            parsed_json = json.loads(json_str)

            # ✅ POST-PROCESS: JSON'ı düzelt
            print("🔧 JSON post-processing yapılıyor...")
            parsed_json = post_process_json(parsed_json)

            task_count = len(parsed_json.get("tasks", []))
            epics = set([task.get("epic_name") for task in parsed_json.get("tasks", [])])

            print(f"✅ AŞAMA 2 (MODEL + PARSE) tamamlandı: {task_count} task üretildi ({len(epics)} epic)")
            return parsed_json, None
        else:
            return None, "JSON formatı bulunamadı"
    except json.JSONDecodeError as e:
        # Hata durumunda JSON'ı dosyaya kaydet
        with open("error_json.txt", "w", encoding="utf-8") as f:
            f.write(json_str)
        print(f"❌ Hatalı JSON kaydedildi: error_json.txt")
        return None, f"JSON parse hatası: {str(e)}"

def parse_json_and_create_prompt(json_input):
    """JSON girdisini parse edip user promptunu oluşturur (YENİ FORMAT)"""
    try:
        data = json.loads(json_input) if isinstance(json_input, str) else json_input

        # Yeni alanları parse et
        project_title = data.get("project_title", "Proje")
        project_desc = data.get("project_description", "").strip().replace('*', '') # Bold'u temizle
        possible_solution = data.get("possible_solution", "").strip()
        estimated_time = data.get("estimated_time", "Belirtilmemiş")

        # Metadata bilgilerini al
        metadata = data.get("metadata", {})
        description = metadata.get("description", "")
        company = metadata.get("company", "")
        department = metadata.get("department", "")
        year = metadata.get("year", 2025)
        # languages alanını prompt'a eklemeye gerek yok, detaylı bilgiye odaklanalım

        # --- Proje Bilgilerini Oluştur ---
        project_info = f"**Proje Adı:** {project_title}\n"
        project_info += f"**Tahmini Süre:** {estimated_time}\n\n"

        if description:
            project_info += f"**Kısa Açıklama:** {description}\n\n"

        if company or department:
            project_info += f"**Şirket/Departman:** {company} - {department}\n"
            project_info += f"**Yıl:** {year}\n\n"

        project_info += f"**Proje Dokümanı (Detaylı Açıklama):**\n{project_desc}\n\n"

        # Eğer olası çözüm varsa ekle
        if possible_solution:
            project_info += f"** Tercihler:**\n{possible_solution}\n\n"

        # --- Ekip Bilgilerini Oluştur ---
        team_info = "**Ekip Üyeleri (Yetenek ve Kapasite Bilgileri):**\n"
        for member in data.get("team", []):
            name = member.get("name", "")
            employee_id = member.get("employee_id", "")
            role = member.get("role", "Belirtilmemiş Rol") # Yeni alan
            capacity = member.get("capacity_hours_per_week", 0) # Yeni alan
            load = member.get("current_load_hours", 0) # Yeni alan
            timezone = member.get("timezone", "N/A") # Yeni alan

            # Yetenekleri (skills) yeni obj formatından al ve seviye ile birleştir
            skill_list = []
            for skill_item in member.get("skills", []):
                skill_name = skill_item.get("name", "")
                skill_level = skill_item.get("level", 0)
                if skill_name:
                     skill_list.append(f"{skill_name} (Lv{skill_level})")

            skills_str = ", ".join(skill_list)

            team_info += (
                f"- **{name}** ({employee_id})\n"
                f"  - **Rol:** {role}\n"
                f"  - **Yetkinlikler:** {skills_str}\n"
                f"  - **Kapasite:** Haftalık {capacity} saat (Mevcut Yük: {load} saat)\n"
                f"  - **Saat Dilimi:** {timezone}\n"
            )

        if not data.get("team"):
            team_info += "Ekip Üyesi Yok\n"

        user_prompt = project_info + team_info
        return user_prompt, None

    except Exception as e:
        # Hata durumunda, hata mesajını geri dön
        return "", f"JSON Parse Hatası: {str(e)}"

def stage1_generate_tasks(json_input, temperature=0.1, max_tokens=3000):
    """AŞAMA 1: JSON'dan Epic/Task planı üret"""
    print("\n🔄 AŞAMA 1: Task Planlayıcı çalışıyor...")

    user_prompt, error = parse_json_and_create_prompt(json_input)
    if error:
        return None, error

    sampling_params1 = SamplingParams(
        temperature=temperature,
        top_p=0.95,
        max_tokens=3000
    )

    formatted_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM_PROMPT_PLANNER}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    outputs = llm_planner.generate([formatted_prompt], sampling_params1)

    response = outputs[0].outputs[0].text

    print("✅ AŞAMA 1 tamamlandı: Epic/Task planı oluşturuldu")
    return response, None

def split_tasks_into_chunks(task_text, chunk_size=7):
    """Task'ları N'şerli gruplara böl (Epic'ler korunarak)"""
    lines = task_text.split('\n')

    chunks = []
    current_chunk = []
    current_epic = None
    task_count_in_chunk = 0

    for line in lines:
        # Epic başlığı
        if line.startswith('## '):
            current_epic = line
            current_chunk.append(line)

        # Task başlığı
        elif line.startswith('### '):
            # Eğer chunk dolmuşsa, yeni chunk başlat
            if task_count_in_chunk >= chunk_size and current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = [current_epic] if current_epic else []
                task_count_in_chunk = 0

            current_chunk.append(line)
            task_count_in_chunk += 1

        # Task içeriği
        else:
            current_chunk.append(line)

    # Son chunk'ı ekle
    if current_chunk:
        chunks.append('\n'.join(current_chunk))

    return chunks

def count_tasks_in_markdown(task_text):
    """Markdown'daki toplam task sayısını say"""
    return task_text.count('### ')

def merge_jira_jsons(json_list):
    """Birden fazla Jira JSON'ı birleştir"""
    merged_tasks = []

    for jira_json in json_list:
        if jira_json and "tasks" in jira_json:
            merged_tasks.extend(jira_json["tasks"])

    return {"tasks": merged_tasks}

def full_pipeline(json_input, project_key="INSIGHT", use_model=True):
    """
    Tam pipeline: JSON → Epic/Task → Jira JSON

    Args:
        json_input: Proje bilgileri JSON'u
        project_key: Jira proje kodu
        use_model: True ise Model + Parse (Model JSON üretir, sonra parse eder)
                   False ise Model + Direkt Parse (Model kullanmadan parse)
    """
    print("\n" + "="*60)
    print(f"🚀 PIPELINE BAŞLATILDI (Method: {'MODEL+PARSE' if use_model else 'PARSE'})")
    print("="*60)

    # Aşama 1: Epic/Task planı oluştur (DPO Model)
    task_text, error1 = stage1_generate_tasks(json_input)

    if error1:
        return {"error": error1, "stage": "stage1"}

    # Task sayısını kontrol et
    total_tasks = count_tasks_in_markdown(task_text)
    print(f"\n📊 Toplam Task Sayısı: {total_tasks}")

    # Task sayısı 7'den fazlaysa chunk'lara böl
    if total_tasks > 7:
        print(f"⚠️ Task sayısı 7'den fazla! {total_tasks} task, 7'şerli gruplara bölünüyor...")
        task_chunks = split_tasks_into_chunks(task_text, chunk_size=7)
        print(f"📦 {len(task_chunks)} grup oluşturuldu")

        # Her chunk için JSON üret
        all_jsons = []
        for i, chunk in enumerate(task_chunks, 1):
            chunk_tasks = count_tasks_in_markdown(chunk)
            print(f"\n🔄 Grup {i}/{len(task_chunks)} işleniyor ({chunk_tasks} task)...")

            if use_model:
                jira_json, error2 = stage2_convert_to_json_with_model(project_key, chunk)
                if error2:
                    print(f"⚠️ Grup {i} hatası: {error2}")
                    continue
            else:
                try:
                    jira_json = convert_markdown_to_jira_json(project_key, chunk)
                except Exception as e:
                    print(f"⚠️ Grup {i} parse hatası: {str(e)}")
                    continue

            all_jsons.append(jira_json)
            print(f"✅ Grup {i} tamamlandı")

            # GPU belleğini temizle
            torch.cuda.empty_cache()

        # Tüm JSON'ları birleştir
        print(f"\n🔗 {len(all_jsons)} JSON birleştiriliyor...")
        jira_json = merge_jira_jsons(all_jsons)

        final_task_count = len(jira_json.get("tasks", []))
        print(f"✅ Birleştirme tamamlandı: {final_task_count} task")

    else:
        # 7 veya daha az task varsa normal işlem
        print(f"ℹ️ Task sayısı 7 veya daha az, tek seferde işleniyor...")

        if use_model:
            jira_json, error2 = stage2_convert_to_json_with_model(project_key, task_text)
            if error2:
                return {
                    "error": error2,
                    "stage": "stage2_model",
                    "task_plan": task_text
                }
        else:
            try:
                jira_json = convert_markdown_to_jira_json(project_key, task_text)

                task_count = len(jira_json.get("tasks", []))
                epics = set([task.get("epic_name") for task in jira_json.get("tasks", [])])

                print(f"✅ AŞAMA 2 (DIREKT PARSE) tamamlandı: {task_count} task üretildi ({len(epics)} epic)")
            except Exception as e:
                return {
                    "error": f"Parse hatası: {str(e)}",
                    "stage": "stage2_parse",
                    "task_plan": task_text
                }

    print("\n" + "="*60)
    print("✅ PIPELINE TAMAMLANDI")
    print("="*60)

    return {
        "success": True,
        "method": "model+parse" if use_model else "direct_parse",
        "task_plan": task_text,
        "jira_json": jira_json,
        "total_tasks": len(jira_json.get("tasks", []))
    }

# Flask API
app = Flask(__name__)

@app.route('/api/generate', methods=['POST'])
def generate():
    try:
        data = request.get_json()

        if not data:
            return jsonify({"error": "JSON girdisi bulunamadı"}), 400

        json_input = data.get("json_input")
        project_key = data.get("project_key", "INSIGHT")
        use_model = data.get("use_model", True)  # Yeni parametre

        if not json_input:
            return jsonify({"error": "json_input alanı gerekli"}), 400

        result = full_pipeline(json_input, project_key, use_model)
        torch.cuda.empty_cache()

        if "error" in result:
            return jsonify(result), 400

        return jsonify(result), 200

    except Exception as e:
        print(f"❌ Hata: {str(e)}")
        return jsonify({"error": str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "healthy", "models": "loaded"}), 200

# Ngrok başlat
print("\n" + "="*60)
print("🌐 NGROK BAŞLATILIYOR...")
print("="*60)

try:
    public_url = ngrok.connect(5000)
    print(f"\n✅ API Hazır!")
    print(f"📡 Ngrok URL: {public_url}")
    print(f"🔗 API Endpoint: {public_url}/api/generate")
    print(f"💚 Health Check: {public_url}/health")

    print("\n📖 KULLANIM ÖRNEĞİ:")
    print(f"""
import requests

# Yöntem 1: DIREKT PARSE (En Hızlı ve Güvenilir)
# Aşama 1: DPO Model → Epic/Task
# Aşama 2: Parse → JSON (Model kullanmadan)
payload = {{
    "json_input": {{
        "project_description": "E-Ticaret platformu geliştirme...",
        "team": [
            {{"employee_id": "e14", "name": "Ahmet", "skills": ["python"], "department": "Backend"}}
        ]
    }},
    "project_key": "INSIGHT",
    "use_model": False  # Direkt Parse (default)
}}

# Yöntem 2: MODEL + PARSE (Daha Esnek)
# Aşama 1: DPO Model → Epic/Task
# Aşama 2: Instruct Model → JSON
# Aşama 3: Parse → JSON düzeltme (tarih/süre/öncelik formatları)
payload_model = {{
    "json_input": {{
        "project_description": "E-Ticaret platformu geliştirme...",
        "team": [
            {{"employee_id": "e14", "name": "Ahmet", "skills": ["python"], "department": "Backend"}}
        ]
    }},
    "project_key": "INSIGHT",
    "use_model": True  # Model ile JSON üret + Parse ile düzelt
}}

response = requests.post("{public_url}/api/generate", json=payload)
print(response.json())
""")

except Exception as e:
    print(f"❌ Ngrok hatası: {e}")
    print("⚠️ Lütfen ngrok authtoken'ınızı ayarlayın:")
    print("!ngrok config add-authtoken YOUR_TOKEN")

# Flask başlat
print("\n" + "="*60)
print("🚀 SERVER BAŞLATILIYOR...")
print("="*60)

app.run(host='0.0.0.0', port=5000)

CUDA mevcut: True
GPU: NVIDIA A100-SXM4-80GB
GPU Bellek: 79.32 GB
Model 1 yükleniyor (Task Planner)...
INFO 10-25 17:46:57 [utils.py:233] non-default args: {'trust_remote_code': True, 'max_model_len': 4096, 'gpu_memory_utilization': 0.45, 'disable_log_stats': True, 'model': 'ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-25 17:46:58 [model.py:547] Resolved architecture: LlamaForCausalLM
INFO 10-25 17:46:58 [model.py:1510] Using max model len 4096
INFO 10-25 17:46:58 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 10-25 17:47:44 [llm.py:306] Supported_tasks: ['generate']
✅ Model 1 yüklendi!
Model 2 yükleniyor (JSON Converter)...
INFO 10-25 17:47:44 [utils.py:233] non-default args: {'trust_remote_code': True, 'max_model_len': 4096, 'gpu_memory_utilization': 0.45, 'disable_log_stats': True, 'model': 'ytu-ce-cosmos/Turkish-Llama-8b-Instruct-v0.1'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-25 17:47:45 [model.py:547] Resolved architecture: LlamaForCausalLM
INFO 10-25 17:47:45 [model.py:1510] Using max model len 4096
INFO 10-25 17:47:45 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 10-25 17:48:30 [llm.py:306] Supported_tasks: ['generate']
✅ Model 2 yüklendi!

🌐 NGROK BAŞLATILIYOR...

✅ API Hazır!
📡 Ngrok URL: NgrokTunnel: "https://0bd347fd59ea.ngrok-free.app" -> "http://localhost:5000"
🔗 API Endpoint: NgrokTunnel: "https://0bd347fd59ea.ngrok-free.app" -> "http://localhost:5000"/api/generate
💚 Health Check: NgrokTunnel: "https://0bd347fd59ea.ngrok-free.app" -> "http://localhost:5000"/health

📖 KULLANIM ÖRNEĞİ:

import requests

# Yöntem 1: DIREKT PARSE (En Hızlı ve Güvenilir)
# Aşama 1: DPO Model → Epic/Task
# Aşama 2: Parse → JSON (Model kullanmadan)
payload = {
    "json_input": {
        "project_description": "E-Ticaret platformu geliştirme...",
        "team": [
            {"employee_id": "e14", "name": "Ahmet", "skills":

...

✅ AŞAMA 1 tamamlandı: Epic/Task planı oluşturuldu

📊 Toplam Task Sayısı: 6
ℹ️ Task sayısı 7 veya daha az, tek seferde işleniyor...

🔄 AŞAMA 2 (MODEL): JSON Dönüştürücü çalışıyor...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

INFO:werkzeug:127.0.0.1 - - [25/Oct/2025 17:51:47] "POST /api/generate HTTP/1.1" 200 -



📄 HAM JSON ÇIKTI (parse edilmeden):

{
  "tasks": [
    {
      "epic_name": "Frontend Geliştirme",
      "fields": {
        "project": {"key": "INSIGHT"},
        "issuetype": {"name": "Task"},
        "summary": "React ve Next.js ile Temel Web Sitesi Oluşturma",
        "description": {
          "type": "doc",
          "version": 1,
          "content": [
            {
              "type": "paragraph",
              "content": [
                {"type": "text", "text": "Proje için temel bir web sitesi oluşturmak amacıyla React ve Next.js kullanılarak ön uç geliştirme yapın. Bu görev, site için temel yapıyı oluşturacak ve gerekli modülleri kuracaktır."}
              ]
            }
          ]
        },
        "assignee": {
          "name": "Alex Kumar",
          "accountId": "e05"
        },
        "priority": {"name": "High"},
        "duedate": "2023-03-15",
        "labels": ["frontend", "react", "nextjs"],
        "parent": {"key": "FRONTEND-GELİŞTİRME"},
        "time